In [ ]:
# Listando os arquivos de trabalho necessários e anotando problemas de limpeza necessários

Files = [
        'DTB_BRASIL_MUNICIPIO.xls',
        'Areas de Risco_Domicilios RAW.xlsx', # associar COD_BATER ao cod. municipal padrão IBGE e adicionar coluna com nome de municipios
        'Areas de Risco_Pessoas RAW.xls', # associar COD_BATER ao cod. municipal padrão IBGE e adicionar coluna com nome de municipios
        'Mapa INSAN Raw.xlsx', #slugify nomes de municipios
        'População.xlsx', # Municipios repetidos, filtrar por cor, sexo e situação domic. em dfs diferentes?
        'Educação.xlsx', #slugify nomes de municipios
        'Renda e Ocupação.xlsx', # Municipios repetidos, filtrar por cor, sexo e situação domic. em dfs diferentes?
        'Pobreza e Vulnerabilidade.xlsx', # Municipios repetidos, filtrar por cor, sexo e situação domic. em dfs diferentes?
        'Aglomerados Subnormais.xlsx', #slugify nomes de municipios

        '% Pobres Municipal BR 1991-2010.csv', #slugify nomes de municipios
        '% de Extremamente Pobres Municipal BR 1991-2010.csv', #slugify nomes de municipios
        '% Crianças Pobres Censos (91-2010) Municípios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias
        '% Pobres - linha de pobreza baseada em necessidades calóricas - Estadual BR 1976-2014.csv', #slugify UFs
        '% Pobreza e Renda Pobreza Municípios Censos (91-2010) AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias
        'Dados Renda Detalhados Censos (91-2010) Municípios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias
        'Renda Domiciliar Estratificada Censos (91-2010) Municípios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias
        'Estratificação Renda Censos (91-2010) Municípios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias

        'PIB Municipal 1970.csv', #slugify nomes de municipios, checar inexistência de Municipios
        'PIB Municipal 1996.csv', #slugify nomes de municipios, checar inexistência de Municipios
        'PIB Municipal 2000.csv', #slugify nomes de municipios, checar inexistência de Municipios
        'PIB Municipal 2010.csv', #slugify nomes de municipios, checar inexistência de Municipios
        'PIB Municipal 2020.csv', #slugify nomes de municipios, checar inexistência de Municipios

        '% População com acesso a infraestrutura Censos (91-2010) Municipios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias
        '% População com acesso a saneamento SNIS (2017) Municípios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias
        '% População vivendo em domicílios precários Censos (91-2010) Municípios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias

        'Dados Educação Censos (1991-2010) por Sexo, Cor e Modo de Vida Municípios AL BA PE.xlsx', # Nomes municipios com (UF) e colunas vazias
        'Medidas de Desigualdade Censos (91-2010) Municípios AL BA PE.xlsx' # Nomes municipios com (UF) e colunas vazias
        ]

In [ ]:
# Cria função para digestão de arquivos e armazenamento de dfs nos dicionários do projeto

''' DOC:
Lê e armazena arquivos de dados com pd.read_ listados em uma lista passada como argumento.
Distingue entre formatos de arquivo .csv ou .xlsx/.xls. 
Arquivos .xls são lidos e armazenados sem intervenções. 
Em arquivos .csv é identificado o tipo de encoding utilizado com chardet.detect() e o arquivo é lido e tem os separadores substituídos por ',' antes do armazenamento.
Gera nomes dinamicamente, padronizando texto removendo espaços e hífens.
Armazena df em variável no escopo global com globals().
Adiciona arquivos nos dicionários do projeto.

Argumento obrigatório:
Files: Lista com nomes de arquivos para leitura.
'''

def digest_files(Files): 

    # Gerando listas vazia para armazenar com os dfs criados, nomes dos dfs criados e nomes dos arquivos importados
    Lista_dfs            = []
    Lista_nomes_dfs      = []
    Lista_imported_files = []
    
    for file in  Files: 

        # Detecta formato de arquivo.csv          
        if '.csv' in file: 

            # Produz nomes dinamicamente para o df a partir do arquivo 
            name = file.replace('.csv', '').replace(' ', '_').replace('-', '_')
            df   = f'raw_{name}'

            # Tenta criar o df e armazenar em var globalmente a partir de .csv sem intervenções
            try: 
                globals()[df] = pd.read_csv(file)
            
            # Em caso de erro tenta especificar encoding e substituir separadores específicos por ','
            except: 
                try: 
                    # Verifica o tipo de encoding do arquivo .csv e armazena em variável para uso na função pd.read_csv
                    with open (file, 'rb') as f: 
                        encod_result = chardet.detect(f.read())['encoding']

                    # abre o arquivo e substitui os separadores em cada linha antes de pd.read_csv, adiciona o tipo de enconding na função
                    globals()[df] = pd.read_csv(StringIO(''.join(l.replace('\t', ',').replace(';', ',') for l in open(file))), encoding = encod_result)

                    # Adiciona df criado nas listas da função
                    Lista_dfs.append(globals()[df])
                    Lista_nomes_dfs.append(df)
                    Lista_imported_files.append(file)
                    print(f'{df} criado com sucesso a partir do arquivo {file}, encoding detectado: {encod_result}')                      

                # Em caso de persistência de erro, não interrompe o processo e armazena arquivo não processado na lista de erros
                except: 
                    print(f'Impossível criar df a partir de {file}')
                    Errors_list.append(file)

            else: 
                # Adiciona df criado nas listas da função
                Lista_dfs.append(globals()[df])
                Lista_nomes_dfs.append(df)
                Lista_imported_files.append(file)
                print(f'{df} criado com sucesso a partir do arquivo {file}')        

        # Detecta formato de arquivo.xls ou .xlsx
        elif '.xlsx' or '.xls' in file: 
            
            # Produz nomes dinamicamente e cria variável que armazena o conteúdo do arquivo em df no escopo global
            name = file.replace('.xlsx', '').replace('.xls', '').replace(' ', '_').replace('-','_')
            df   = f'raw_{name}'

            try: 
                globals()[df] = pd.read_excel(file)
            except: 
                print(f'Impossível criar df a partir de {file}')
                Errors_list.append(file)
            else: 
                # Adiciona df criado nas listas da função
                Lista_dfs.append(globals()[df])
                Lista_nomes_dfs.append(df)
                Lista_imported_files.append(file)
                print(f'{df} criado com sucesso a partir do arquivo {file}')       

        else: 
            print(f'{file} não é um arquivo .csv, .xls ou .xlsx')

    # Zipa listas da função em dicionários utilizados para atualizar os dicionários do projeto
    dict_update_line = dict(zip(Lista_nomes_dfs, Lista_imported_files))
    Original_Files.update(dict_update_line)
    dict_update_line = dict(zip(Lista_nomes_dfs, Lista_dfs))
    Raw_DFs.update(dict_update_line)
    

In [ ]:
# Digerindo os arquivos listados
digest_files(Files)


In [ ]:
# Verificação
Raw_DFs.keys(), Original_Files